In [2]:
import torch
import time
import os
import numpy as np
import cv2
import tkinter as tk
from tkinter import ttk
from PIL import ImageTk
from PIL import Image
from torchvision import transforms

#Список настраиваемых глобальных переменных(Настройки)
global classes
global model_name
global cam_num
global britness
global theme
global model


global camera_error_flag #Флаг ошибки камеры

camera_error_flag=0

fl=os.path.isfile('user_settings.txt') #Проверка наличия файла пользовательских настроек

if fl == 1:
    #print('файл нашёл')
    f = open('user_settings.txt','r')
    
    theme = f.readline()
    cam_num = f.readline()
    britness = f.readline()
    classes = f.readline()
    model_name = f.readline()

    model_name = model_name.rstrip()
    theme=theme.rstrip()
    cam_num = int(cam_num)
    britness = int(britness)
    
    cap = cv2.VideoCapture(cam_num)
    ret, frame = cap.read()
    if ret == 0:
        camera_error_flag=1
    classes = classes.split(",")
    
    f.close()
else:
    print('файл не нашёл')
    classes = str('dht11,hc-sr04,mpu6050,mq5') # названия классов
    model_name = str('Sensors_net2.pt')
    britness = 100
    cam_num=1
    theme='myclassic'
    
    #Проверка подключения камеры к компьютеру
    
    cap = cv2.VideoCapture(cam_num)
    ret, frame = cap.read()
    if ret == 0:
    
        cam_num=0 #Если подключена одна камера (значение=0)
    
    cap = cv2.VideoCapture(cam_num)
    ret, frame = cap.read()
    if ret == 0:
        camera_error_flag=1 # Флаг камера не подключена
        
    classes = classes.split(",")
    

#Загрузка модели
model = torch.jit.load(model_name)

transform = transforms.Compose(
    [transforms.ToPILImage(),
    transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# Инференс
def clicked():
    global classes
    ret, frame = cap.read()
    if ret == 0:
        lbl2.configure(text='Ошибка захвата кадра',fg='red')
    else:
        img = frame
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = transform(img)
        model.eval()
        img=torch.unsqueeze(img, 0)
        outputs = model(img)
        _, preds = torch.max(outputs, 1)
        lbl2.configure(text='Класс объекта: {}'.format(classes[int(preds)]))
        
def set_settings(): #Сохранение и применение настроек
    
    global model
    
    global theme
    global cam_num
    global britness
    global classes
    global model_name
    
    cam_num=spin.get()
    classes = txt_classnames.get()
    model_name = txt_modelname.get()
    britness=txt_britness.get()
    
    fl=os.path.isfile(model_name)
    if fl == 1:
        #print('файл нашёл')
        del model
        model = torch.jit.load(model_name)
    else:
        txt_modelname.delete(0,tk.END)
        txt_modelname.insert(0,'Нет модели')

    f = open('user_settings.txt', 'w')
    f.write("{}\n{}\n{}\n{}\n{}".format(theme,cam_num,britness,classes.rstrip(),model_name))
    f.close()
    
def reset_settings():
    
    global model
    global theme
    global cam_num
    global britness
    global classes
    global model_name
    
    theme='myclassic'
    cam_num=1
    britness=100
    classes = str('dht11,hc-sr04,mpu6050,mq5')
    model_name = str('Sensors_net2.pt')
    
    fl=os.path.isfile(model_name)
    if fl == 1:
        del model
        model = torch.jit.load(model_name)
    else:
        txt_modelname.delete(0,tk.END)
        txt_modelname.insert(0,'Нет модели') 
    
    f = open('user_settings.txt', 'w')
    f.write("{}\n{}\n{}\n{}\n{}".format(theme,cam_num,britness,classes.rstrip(),model_name))
    f.close()
    
    txt_britness.delete(0,tk.END)
    txt_classnames.delete(0,tk.END)
    txt_modelname.delete(0,tk.END)
    
    txt_britness.insert(0,britness)
    txt_classnames.insert(0,classes)
    txt_modelname.insert(0,model_name)
  
    
        
# Обработка смены цвета темы
def set_classic():
    global theme
    theme='myclassic'
    style.theme_use(theme)
    
    lmain.configure(bg="steel blue",fg="white")
    btn.configure(bg="steel blue",fg="ghost white",activebackground = "steel blue")
    lbl2.configure(bg="dim gray",fg="DarkGoldenrod1")
    
    lbl_colortheme.configure(bg="dim gray",fg="DarkGoldenrod1")
    btn_classic.configure(bg="steel blue",fg="ghost white",activebackground = "steel blue")
    btn_dark.configure(bg="steel blue",fg="ghost white",activebackground = "steel blue")
    btn_light.configure(bg="steel blue",fg="ghost white",activebackground = "steel blue")
    
    lbl_videon.configure(bg="dim gray",fg="DarkGoldenrod1")
    spin.configure(bg='dim gray',fg='DarkGoldenrod1')
    
    lbl_britness.configure(bg="dim gray",fg="DarkGoldenrod1")
    
    lbl_classnames.configure(bg="dim gray",fg="DarkGoldenrod1")
    lbl_modelname.configure(bg="dim gray",fg="DarkGoldenrod1")
    
    btn_set.configure(bg="gray",fg="black")
    btn_reset.configure(bg="gray",fg="black")
    
    text_widget.configure(bg="dim gray",fg="ghost white")
    
    
    
def set_dark():
    global theme
    theme='mydark'
    style.theme_use(theme)
    
    lmain.configure(bg="Gray26",fg="white")
    btn.configure(bg="Gray26",fg="ghost white",activebackground = "Gray26")
    lbl2.configure(bg="Gray36",fg="ghost white")
    
    lbl_colortheme.configure(bg="Gray36",fg="ghost white")
    btn_classic.configure(bg="Gray26",fg="ghost white",activebackground = "Gray26")
    btn_dark.configure(bg="Gray26",fg="ghost white",activebackground = "Gray26")
    btn_light.configure(bg="Gray26",fg="ghost white",activebackground = "Gray26")
    
    lbl_videon.configure(bg="dim gray",fg="ghost white")
    spin.configure(bg='dim gray',fg='black')
    
    lbl_britness.configure(bg="Gray36",fg="ghost white")
    
    lbl_classnames.configure(bg="Gray36",fg="ghost white")
    lbl_modelname.configure(bg="Gray36",fg="ghost white")
    
    btn_set.configure(bg="gray",fg="black")
    btn_reset.configure(bg="gray",fg="black")
    
    text_widget.configure(bg="Gray36",fg="ghost white")
    
def set_light():
    global theme
    theme='mylight'
    style.theme_use(theme)
    
    lmain.configure(bg="gainsboro",fg="black")
    btn.configure(bg="gainsboro",fg="black",activebackground = "gainsboro")
    lbl2.configure(bg='snow2',fg="black")
    
    lbl_colortheme.configure(bg='snow2',fg="black")
    btn_classic.configure(bg="gainsboro",fg="black",activebackground = "gainsboro")
    btn_dark.configure(bg="gainsboro",fg="black",activebackground = "gainsboro")
    btn_light.configure(bg="gainsboro",fg="black",activebackground = "gainsboro")
    
    lbl_videon.configure(bg='snow2',fg="black")
    spin.configure(bg='snow2',fg='black')
    
    lbl_britness.configure(bg='snow2',fg="black")
    
    lbl_classnames.configure(bg='snow2',fg="black")
    lbl_modelname.configure(bg='snow2',fg="black")
    
    btn_set.configure(bg="light gray",fg="black")
    btn_reset.configure(bg="light gray",fg="black")
    
    text_widget.configure(bg='snow2',fg="black")
    

#Графический интерфейс
window = tk.Tk()  #главное окно

#Стили графического интерфейса
style = ttk.Style()

#Классическая тема
style.theme_create('myclassic', settings={
    'TLabel': {'configure': {'background': 'white'}},
    'TFrame': {'configure': {'background': 'dim gray'}},
    'TNotebook': {
        'configure': {'background': "Gray17", 'tabmargins': [0, 7, 0, 2], 'padding': [10, 2]}},
    'TNotebook.Tab': {
        'configure': {'foreground':"ghost white",'background': "steel blue", 'padding': [16, 2],
                      'focuscolor':"dim gray",'font':("Arial Bold", 30)},
       'map': {'background': [('selected', 'dim gray')]}}})

#Тёмная тема
style.theme_create('mydark', settings={
    'TLabel': {'configure': {'background': 'Gray36'}},
    'TFrame': {'configure': {'background': 'Gray36'}},
    'TNotebook': {
        'configure': {'background': "Gray10", 'tabmargins': [0, 7, 0, 2], 'padding': [10, 2]}},
    'TNotebook.Tab': {
        'configure': {'foreground':"ghost white",'background': "Gray26", 'padding': [16, 2],
                      'focuscolor':'Gray36','font':("Arial Bold", 30)},
       'map': {'background': [('selected', 'Gray36')]}}})

#Светлая тема
style.theme_create('mylight', settings={
    'TLabel': {'configure': {'background': 'white'}},
    'TFrame': {'configure': {'background': 'snow2'}},
    'TNotebook': {
        'configure': {'background': "Gray63", 'tabmargins': [0, 7, 0, 2], 'padding': [10, 2]}},
    'TNotebook.Tab': {
        'configure': {'foreground':"black",'background': "gainsboro", 'padding': [16, 2],
                      'focuscolor':'snow2','font':("Arial Bold", 30)},
       'map': {'background': [('selected', 'snow2')]}}})


style.theme_use(theme) #Установка стиля

window.wm_title("Классификатор")
window.configure(background="dim gray")
tab_control = ttk.Notebook(window)
tab1 = ttk.Frame(tab_control)
tab2 = ttk.Frame(tab_control)
tab3 = ttk.Frame(tab_control)
tab_control.add(tab1, text='Главная')  
tab_control.add(tab2, text='Настройки')
tab_control.add(tab3, text='Инструкция')

#Главная вкладка(Начало)

#Окно вывода с камеры

lmain = tk.Label(tab1,background="steel blue",fg="white")
lmain.grid(row=0, column=0,padx=10, pady=10)
lmain.pack

cap = cv2.VideoCapture(cam_num)

#cap.set(3,224) #Ширина
#cap.set(4,224) #Высота
cap.set(10,britness) #Яркость

    #Виджет с выводом названия класса
    
class_name = '' 
lbl2 = tk.Label(tab1, text=class_name,
                font=("Times New Roman", 30),
                bg="dim gray",
                fg="DarkGoldenrod1")
lbl2.pack
lbl2.grid(column=0, row=1,padx=10, pady=10)

    #Кнопка распознать

btn = tk.Button(tab1, text="Распознать",
                font=("Arial Bold", 30),
                bg="steel blue",
                fg="ghost white",
                command=clicked)
btn.pack
btn.grid(column=0, row=2,padx=10, pady=10)

#Главная вкладка(Конец)


#Вкладка настройки(Начало)
lbl_colortheme=tk.Label(tab2, text="Цветовая тема",
                font=("Times New Roman", 30),
                bg="dim gray",
                fg="DarkGoldenrod1")
lbl_colortheme.grid(column=0,row=0,columnspan=3,padx=10, pady=10)

btn_classic=tk.Button(tab2,text="Классическая",
                font=("Arial Bold", 25),
                bg="steel blue",
                fg="ghost white",
                command=set_classic)
btn_classic.grid(column=0, row=1,padx=10, pady=10)

btn_dark=tk.Button(tab2,text="Тёмная",
                font=("Arial Bold", 25),
                bg="steel blue",
                fg="ghost white",
                command=set_dark)
btn_dark.grid(column=1, row=1,padx=10, pady=10)

btn_light=tk.Button(tab2,text="Светлая",
                font=("Arial Bold", 25),
                bg="steel blue",
                fg="ghost white",
                command=set_light)
btn_light.grid(column=2, row=1,padx=10, pady=10)

lbl_videon=tk.Label(tab2,text="Номер порта камеры",
                font=("Times New Roman", 30),
                bg="dim gray",
                fg="DarkGoldenrod1")
lbl_videon.grid(column=0,row=2,columnspan=2,padx=10, pady=10)

var = tk.StringVar()
var.set(str(cam_num))

spin = tk.Spinbox(tab2, from_=0, to=5, width=2,textvariable=var,
    bg='dim gray',fg='DarkGoldenrod1',font=("Times New Roman", 30))

spin.grid(column=2, row=2)

lbl_britness=tk.Label(tab2,text="Яркость камеры",
                font=("Times New Roman", 30),
                bg="dim gray",
                fg="DarkGoldenrod1")

lbl_britness.grid(column=0,row=3,columnspan=2,padx=10, pady=10)
txt_britness=tk.Entry(tab2,width=10,font=("Times New Roman", 30))
txt_britness.grid(column=2,row=3,padx=10, pady=10)

lbl_classnames=tk.Label(tab2,text="Имена классов",
                font=("Times New Roman", 30),
                bg="dim gray",
                fg="DarkGoldenrod1")
lbl_classnames.grid(column=0,row=4,columnspan=2,padx=10, pady=10)
txt_classnames = tk.Entry(tab2,width=10,font=("Times New Roman", 30))
txt_classnames.grid(column=2,row=4,padx=10, pady=10)

lbl_modelname=tk.Label(tab2,text="Файл модели",
                font=("Times New Roman", 30),
                bg="dim gray",
                fg="DarkGoldenrod1")
lbl_modelname.grid(column=0,row=5,columnspan=2,padx=10, pady=10)
txt_modelname = tk.Entry(tab2,width=10,font=("Times New Roman", 30))
txt_modelname.grid(column=2,row=5,padx=10, pady=10)

txt_britness.insert(0,britness)
txt_classnames.insert(0,classes) 
txt_modelname.insert(0,model_name)

btn_set=tk.Button(tab2,text="Применить",
                font=("Arial Bold", 25),
                bg="gray",
                fg="black",
                command=set_settings)
btn_set.grid(column=2,row=6,padx=10, pady=10)

btn_reset=tk.Button(tab2,text="Сбросить",
                font=("Arial Bold", 25),
                bg="gray",
                fg="black",
                command=reset_settings)
btn_reset.grid(column=0,row=6,padx=10, pady=10)

#Вкладка настройки(Конец)

text_widget=tk.Text(tab3,height=25,width=56,font=("Arial Bold", 14),bg="dim gray",fg="ghost white")
text_widget.grid(column=0,row=0,padx=24, pady=30)
some_text="""О программе

   AI Classifier - программа для классификации изображений на  базе 
 нейросетевых   моделей.  Обученная  нейронная   сеть   определяет 
 прикладную задачу классификации. AI Classifier можно использовать
 для  решения  задач  классификации  изображений  и  как  средство
 проверки  работы   готовой    сети.    AI   Classifier   поддерживает
 сериализованные  модели  нейронных  сетей  PyTorch  в  формате.pt
 
 Как использовать AI Classifier
                        
   Чтобы определить класс объекта, нужно навести камеру на объект
 и  нажать кнопку 'Распознать'. Инференс  производится с помощью 
 текущей  модели  сети.  Модель  должна  находится  в  директории 
 программы и иметь расширение.pt. Чтобы поменять текущую модель
 на свою, нужно переместить сеть  в  папку  программы, поменять
 название  модели  и  список   классов  во   вкладке  настройки,
 названия классов указываются через запятую.

 Настройки
                    
 Вкладка 'Настройки' содержит следующие элементы

 1. Настройка цветовой темы

 Темы: Классическая, Тёмная, Светлая
 Чтобы сохранить настройки следует нажать кнопку применить.

 2. Номер порта камеры

 Номер порта по умолчанию задаётся 1, 
 если подключена внешняя камера и 0,
 если камера не подключена к первому порту.

 Если требуется выбрать конкретную камеру, то следует переключить
 порт.  Изменение будет выполненно после перезапуска программы.

 3. Яркость камеры

 Настройки камер могут отличаться, следует выбрать оптимальное 
 значение. Изменение будет выполненно после перезапуска програм-
 мы.

 4. Имена классов

 В поле вводится список названий классов, которые будут 
 использваться при выводе ответа сети. Имена указываются 
 через запятую.

 5. Файл модели

 В поле вводится название модели с расширением .pt, модель
 должна находится в директории программы

 Изменения 4,5 необходимо применить. Они будут применены сразу.

 Кнопка 'применить' сохраняет все настройки, они будут доступны при 
 следующем запуске. Кнпока 'сбросить' возвращает все настройки к
 начальным значениям.
 
 Автор: Шумачков Антон
"""
text_widget.insert(tk.END,some_text)


#Функция вывода изображения на экран
def show_frame():
    global camera_error_flag
    ret, frame = cap.read()#Захват кадров
    if ret == 1:
        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        img = Image.fromarray(cv2image)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)
        lmain.after(10, show_frame)
if camera_error_flag == 0:
    show_frame()  #Вызов функуции показать изображение
else: 
    lmain.configure(text='Ошибка подключения камеры',font=("Times New Roman", 20),bg="light gray",fg="red")
tab_control.pack(expand=1, fill='both')

if theme == 'myclassic': 
    set_classic()
    #print('classicl')
if theme == 'mydark':
    set_dark()
    #print('dark')
if theme == 'mylight': 
    set_light()
    #print('light')

window.mainloop()  #Запускает графический интерфейс

cap.release()
cv2.destroyAllWindows()
del model

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-2-29b29b6fdb49>", line 96, in clicked
    lbl2.configure(text='Класс объекта: {}'.format(classes[int(preds)]))
IndexError: list index out of range
